In [13]:
import torch

import torch.nn as nn

import torch.nn.functional as F 

import numpy as np

1. Dataset Preparation

In [14]:
# Tiny corpus

corpus = [
    "deep learning is powerful",
    "learning is fun and powerful",
    "deep learning is fun"
]


In [15]:
# Tokenize

tokens = " ".join(corpus).split()

vocab = sorted(set(tokens))

word_to_idx = {word: i for i, word in enumerate(vocab)}

idx_to_word = {i: word for word, i in word_to_idx.items()}

In [16]:
# Convert to sequences

seq_len = 3

data = []

In [17]:
for sentence in corpus:

    words = sentence.split()

    for i in range(len(words) - seq_len):

        input_seq = words[i: i + seq_len]

        target_word = words[i+seq_len]

        input_idx = [word_to_idx[word] for word in input_seq]

        target_idx = word_to_idx[target_word]

        data.append((input_idx, target_idx))


input_seqs = [torch.tensor(x) for x , _ in data]

target_words = [torch.tensor(y) for y ,_ in data]

2. Define LSTM Model

In [18]:
class WordLSTM(nn.Module):

    def __init__(self, vocab_size,embedding_dim, hidden_dim):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):

        x = self.embedding(x) # [batch, seq, embed]

        output, hidden = self.lstm(x, hidden) # [batch, seq, hidden]

        output = self.fc(output[:,-1,:]) # take last time step

        return output, hidden

3. Training Loop

In [19]:
emebedding_dim = 10

hidden_dim = 32

vocab_size = len(vocab)

learning_rate = 0.01

num_epochs = 3.0

In [20]:
model = WordLSTM(vocab_size, emebedding_dim, hidden_dim)

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
